Imports

In [1]:
import logging
import ssl
import urllib3

from elasticsearch import Elasticsearch, JSONSerializer
from elasticsearch.connection import create_ssl_context
from elasticsearch.helpers import parallel_bulk
from elasticsearch.helpers import scan
import numpy as np
import pandas as pd

urllib3.disable_warnings() # This is insecure
logger = logging.getLogger(__name__)

Elastic search connection procedures:

In [2]:
def get_elastic_client(server="local", write=False):
    if server == "local":
        if write:
            serializer = NpJSONSerializer()
        else:
            serializer = JSONSerializer()
        return Elasticsearch(host="localhost",
                             port=9200,
                             serializer=serializer)
    elif server in {"dev", "horizon", "prod"}:
        # All the other servers are remote hosts with similar configs
        if server == "prod":
            host = "daimler-elastic.vpc.bigml.com"
        elif server == "horizon":
            host = "daimler-elastic.horizon.bigml.com"
        else:
            host = "daimler-elastic.dev.bigml.com"

        if write:
            serializer = NpJSONSerializer()
        else:
            serializer = JSONSerializer()
        # Set up ssl context to disable cert verification
        ssl_context = create_ssl_context()
        ssl_context.check_hostname = False
        ssl_context.verify_mode = ssl.CERT_NONE
        return Elasticsearch(host=host,
                             port=443,
                             http_auth=("dev", "paroafCa"),
                             serializer=serializer,
                             ssl_context=ssl_context,
                             use_ssl=True,
                             request_timeout=50,
                             timeout=50,
                             max_retries=5, 
                             retry_on_timeout=True)
    else:
        logger.warning("unknown server '%s'", server)
        return None

Instanciate Elastic Search connection to dev:

In [3]:
ES = get_elastic_client("dev")
print(ES)

<Elasticsearch([{}])>


## 1) Get Tools Welds Counts from RAW index

In [11]:
# DEFINE QUERY
# execute query to get referential welds data
tools_welds_count_query = {
  "size" : 1,
  "_source" : {
    "includes" : [
      "uniqueID",
      "COUNTER.Components.Outlets.WeldTools.Weldtool_WeldsTotal",
      "COUNTER.Components.Outlets.WeldTools.Statistics.WeldCounter_WOPs",
      "timestamp",
      "location",
      "series",
      "line",
      "plant",
      "station",
      "robot",
      "controller",
      "tool"
    ]
  }
}

In [5]:
REQUEST_TIMEOUT = 100.  # Timeout for the elastic queries

In [12]:
repaired_welds_df = pd.DataFrame()

num_batch = 1
welds = []

for current_weld in scan(ES, index="ml_toolbox_counter", 
                                  query=tools_welds_count_query, 
                                  scroll='15m',
                                  raise_on_error=True,
                                  size=5000,
                                  request_timeout=REQUEST_TIMEOUT):

    if num_batch % 10000 == 0:
      print("Current batch %s" % num_batch)
    
    num_batch = num_batch + 1

    # gather current bucket key information
    welds.append(current_weld["_source"])


## build dataframe from welds list
print("Loop finished, building dataframe ")
print(len(welds))

tools_counters_df = pd.DataFrame.from_records(welds)

Current batch 10000
Current batch 20000
Current batch 30000
Current batch 40000
Current batch 50000
Current batch 60000
Current batch 70000
Current batch 80000
Current batch 90000
Current batch 100000
Current batch 110000
Current batch 120000
Current batch 130000
Current batch 140000
Current batch 150000
Current batch 160000
Current batch 170000
Current batch 180000
Current batch 190000
Current batch 200000
Current batch 210000
Current batch 220000
Current batch 230000
Current batch 240000
Current batch 250000
Current batch 260000
Current batch 270000
Current batch 280000
Current batch 290000
Current batch 300000
Current batch 310000
Current batch 320000
Current batch 330000
Current batch 340000
Current batch 350000
Current batch 360000
Current batch 370000
Current batch 380000
Current batch 390000
Current batch 400000
Current batch 410000
Current batch 420000
Current batch 430000
Current batch 440000
Current batch 450000
Current batch 460000
Current batch 470000
Current batch 480000
C

KeyboardInterrupt: 

In [8]:
print(tools_counters_df.shape)
print(tools_counters_df.head())

(7618853, 4)
   COUNTER.Components.Outlets.WeldTools.Statistics.WeldCounter_WOPs  \
0                                                867                  
1                                                  2                  
2                                                527                  
3                                                420                  
4                                                  0                  

                                   uniqueID                        timestamp  \
0  h902-110tsb401-kf130.m050g9sub63sps3.1.1  2019-12-17T17:20:30.529000+0100   
1  h902-140tsb101-kf130.m050g9sub63sps3.1.1  2020-01-08T10:03:00.149000+0100   
2  h902-110tsb201-kf130.m050g9sub63sps3.1.1  2020-02-19T12:55:46.472000+0100   
3  h902-170tsb101-kf130.m050g9sub64sps5.1.1  2020-03-17T14:05:35.730000+0100   
4  h902-140tsb101-kf130.m050g9sub63sps3.2.1  2020-04-27T13:35:27.748000+0200   

   COUNTER.Components.Outlets.WeldTools.Weldtool_WeldsTotal  
0                

In [10]:
tools_counters_df.to_csv('/Users/guillem/Data/Customers/Daimler/health_index/tool_counts_extended.csv', index = False, header=True)